In [2]:
from finance.llm.summarization import TranscriptLoader

In [3]:
dataloader = TranscriptLoader(collection_name = "transcripts_mililm_l6_v3", embedding_model = "all-MiniLM-L6-v2")
dataloader.instantiate_client()

where_dict = {'$and':[
  {'symbol': {
   "$in": ['TSLA']}
  }, 
  {'year': {
"$gt": 2023}
  }]
 }

result = dataloader.query_client('TESLA transcripts', n_results=1, **where_dict)
texts = dataloader.get_texts(result, 1)

/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/finance/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/finance/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


TypeError: TranscriptLoader.get_texts() missing 1 required positional argument: 'preprocess'

In [5]:
from langchain.llms import GPT4All
from langchain.callbacks.streaming_stdout_final_only import (FinalStreamingStdOutCallbackHandler)
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


from langchain.chains import (
    StuffDocumentsChain,
    LLMChain,
    ReduceDocumentsChain,
    MapReduceDocumentsChain,
)
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import OpenAI

# This controls how each document will be formatted. Specifically,
# it will be passed to `format_document` - see that function for more
# details.
document_prompt = PromptTemplate(
    input_variables=["page_content"],
     template="{page_content}"
)
document_variable_name = "context"


callbacks = [FinalStreamingStdOutCallbackHandler()] 
callbacks = [StreamingStdOutCallbackHandler()]
llm = GPT4All(model='../llm_models/gpt4all-falcon-q4_0.gguf', callbacks=callbacks, verbose=True)# The prompt here should take as an input variable the
# `document_variable_name`
prompt = PromptTemplate.from_template(
    """The following is a part of a transcript for a company containing it's financial performance.
        {context}
        Summarize the text focussing on challenges and successes and elaborate on the most important details.
        Helpful Answer:"""
)
llm_chain = LLMChain(llm=llm, prompt=prompt)

# We now define how to combine these summaries
reduce_prompt = PromptTemplate.from_template(
    """You will receive a summarized text with challenges and successes for a company.
        {context}
        Firstly extract the year and quarter for which the transcript is for and list it at the top.
        Secondly based on this set of docs, please summarize and list the following:
        - main challenges
        - main successes
        Helpful Answer:"""
)
reduce_llm_chain = LLMChain(llm=llm, prompt=reduce_prompt)

combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name
)


# collapse_documents_chain which is specifically aimed at collapsing documents BEFORE
# the final call.
prompt = PromptTemplate.from_template(
            "Collapse this content while keeping the most important information: {context}"
        )
llm_chain = LLMChain(llm=llm, prompt=prompt)
collapse_documents_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name
)
reduce_documents_chain = ReduceDocumentsChain(
    combine_documents_chain=combine_documents_chain,
    collapse_documents_chain=collapse_documents_chain,
    token_max=2000
)
chain = MapReduceDocumentsChain(
    llm_chain=llm_chain,
    reduce_documents_chain=reduce_documents_chain,
)

In [6]:
from finance.llm.transcript import TranscriptLoader

dataloader = TranscriptLoader(collection_name = "transcripts_mililm_l6_v3", embedding_model = "all-MiniLM-L6-v2")
dataloader.instantiate_client()
    
where_dict = {'$and':[
              {'symbol': {
                       "$in": ['TSLA']}
              }, 
              {'year': {
                        "$gt": 2023}
              }]
         }
    
result = dataloader.query_client('TESLA transcripts', n_results=1, **where_dict)
texts = dataloader.get_texts(result, 1)

In [7]:
output = chain.invoke(texts)

 to have a fully autonomous factory by the end of next year. We're also making progress on our Giga Berlin factory, which will be the largest factory in the world when it's completed. We're also making progress on our Giga Shanghai factory, which will be the largest factory in the world when it's completed. And we're also making progress on our Giga Austin factory, which will be the largest factory in the world when it's completed. So we're making progress on all of these things. We're also making progress on our Giga Shanghai factory, which will be the largest factory in the world when it's completed. And we're also making progress on our Giga Austin factory, which will be the largest factory in the world when it's completed. So we're making progress on all of these things. We're also making progress on our Giga Shanghai factory, which will be the largest factory in the world when it's completed. And we're also making progress on our Giga Austin factory, which will be the largest fact

/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/finance/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (4371 > 1024). Running this sequence through the model will result in indexing errors


 the technology gets better and better, you're going to see more and more trucks on the road. And I think that's going to be a big driver of adoption. And then there are also things like regulatory factors. So, as governments start to mandate the use of autonomous trucks, you're going to see more and more adoption.
So, I think all of those things will drive adoption over the next few years. And I think on top of. And so, we're going to be able to use our own data and insights to improve our own products and services.
Travis Axelrod: Thank you. Our next question comes from David from JPMorgan. David, please unmute yourself.
David: Hi, thanks for taking my question. I wanted to follow up on the question about the potential for the company to be a platform for other companies to build on top of. Can you talk about how you're thinking about that and what kind of partnerships you might be looking to form in that space?
Elon Musk: Yeah, so we're definitely looking at building out a platform.

In [13]:
from bert_score import BERTScorer
from rouge import Rouge
from rouge_score import rouge_scorer

scorer = BERTScorer(model_type='bert-base-uncased')

orig = result['documents'][0][0]
output = open('output_raw_llm.txt','r').read()
P, R, F1 = scorer.score([orig], [output]) 
P, R, F1

/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/finance/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


(tensor([0.4213]), tensor([0.5115]), tensor([0.4621]))

In [15]:
pd.DataFrame({'Precision':P, 'Recall':R, 'F1':F1}).T

,0
Precision,0.421326
Recall,0.511537
F1,0.462069


In [6]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(orig, output)
print(f"ROUGE-1 Precision: {scores['rouge1'].precision:.4f}, Recall: {scores['rouge1'].recall:.4f}, F1: {scores['rouge1'].fmeasure:.4f}")
print(f"ROUGE-2 Precision: {scores['rouge2'].precision:.4f}, Recall: {scores['rouge2'].recall:.4f}, F1: {scores['rouge2'].fmeasure:.4f}")
print(f"ROUGE-L Precision: {scores['rougeL'].precision:.4f}, Recall: {scores['rougeL'].recall:.4f}, F1: {scores['rougeL'].fmeasure:.4f}")

ROUGE-1 Precision: 0.8246, Recall: 0.0109, F1: 0.0214
ROUGE-2 Precision: 0.2212, Recall: 0.0029, F1: 0.0057
ROUGE-L Precision: 0.5000, Recall: 0.0066, F1: 0.0130


In [8]:
import pandas as pd

In [12]:
# function to calculate the Rouge score
def get_rouge_scores(text1, text2):
    rouge = Rouge()
    return rouge.get_scores(text1, text2)


rouge_scores_out = []

# Calculate the ROUGE scores for both summaries using reference
eval_1_rouge = get_rouge_scores(orig, output)

for metric in ["rouge-1", "rouge-2", "rouge-l"]:
    for label in ['r', 'p', 'f']:
        eval_1_score = eval_1_rouge[0][metric][label[0].lower()]

        row = {
            "Metric": f"{metric} ({label})",
            "Summary 1": eval_1_score,
        }
        rouge_scores_out.append(row)


def highlight_max(s):
    is_max = s == s.max()
    return [
        "background-color: lightgreen" if v else "background-color: white"
        for v in is_max
    ]


rouge_scores_out = (
    pd.DataFrame(rouge_scores_out)
    .set_index("Metric")
)

rouge_scores_out

,Summary 1
Metric,
rouge-1 (r),0.528736
rouge-1 (p),0.023650
rouge-1 (f),0.045276
rouge-2 (r),0.116667
rouge-2 (p),0.002231
rouge-2 (f),0.004379
rouge-l (r),0.482759
rouge-l (p),0.021594
rouge-l (f),0.041339


In [10]:
eval_1_rouge

[{'rouge-1': {'r': 0.5287356321839081,
   'p': 0.02365038560411311,
   'f': 0.045275589731544316},
  'rouge-2': {'r': 0.11666666666666667,
   'p': 0.0022314313037934334,
   'f': 0.004379105043015708},
  'rouge-l': {'r': 0.4827586206896552,
   'p': 0.021593830334190232,
   'f': 0.041338581857528575}}]

In [ ]:
# Evaluation prompt template based on G-Eval
EVALUATION_PROMPT_TEMPLATE = """
You will be given one summary written for an article. Your task is to rate the summary on one metric.
Please make sure you read and understand these instructions very carefully. 
Please keep this document open while reviewing, and refer to it as needed.

Evaluation Criteria:

{criteria}

Evaluation Steps:

{steps}

Example:

Source Text:

{document}

Summary:

{summary}

Evaluation Form (scores ONLY):

- {metric_name}
"""

# Metric 1: Relevance

RELEVANCY_SCORE_CRITERIA = """
Relevance(1-5) - selection of important content from the source. \
The summary should include only important information from the source document. \
Annotators were instructed to penalize summaries which contained redundancies and excess information.
"""

RELEVANCY_SCORE_STEPS = """
1. Read the summary and the source document carefully.
2. Compare the summary to the source document and identify the main points of the article.
3. Assess how well the summary covers the main points of the article, and how much irrelevant or redundant information it contains.
4. Assign a relevance score from 1 to 5.
"""

# Metric 2: Coherence

COHERENCE_SCORE_CRITERIA = """
Coherence(1-5) - the collective quality of all sentences. \
We align this dimension with the DUC quality question of structure and coherence \
whereby "the summary should be well-structured and well-organized. \
The summary should not just be a heap of related information, but should build from sentence to a\
coherent body of information about a topic."
"""

COHERENCE_SCORE_STEPS = """
1. Read the article carefully and identify the main topic and key points.
2. Read the summary and compare it to the article. Check if the summary covers the main topic and key points of the article,
and if it presents them in a clear and logical order.
3. Assign a score for coherence on a scale of 1 to 5, where 1 is the lowest and 5 is the highest based on the Evaluation Criteria.
"""

# Metric 3: Consistency

CONSISTENCY_SCORE_CRITERIA = """
Consistency(1-5) - the factual alignment between the summary and the summarized source. \
A factually consistent summary contains only statements that are entailed by the source document. \
Annotators were also asked to penalize summaries that contained hallucinated facts.
"""

CONSISTENCY_SCORE_STEPS = """
1. Read the article carefully and identify the main facts and details it presents.
2. Read the summary and compare it to the article. Check if the summary contains any factual errors that are not supported by the article.
3. Assign a score for consistency based on the Evaluation Criteria.
"""

# Metric 4: Fluency

FLUENCY_SCORE_CRITERIA = """
Fluency(1-3): the quality of the summary in terms of grammar, spelling, punctuation, word choice, and sentence structure.
1: Poor. The summary has many errors that make it hard to understand or sound unnatural.
2: Fair. The summary has some errors that affect the clarity or smoothness of the text, but the main points are still comprehensible.
3: Good. The summary has few or no errors and is easy to read and follow.
"""

FLUENCY_SCORE_STEPS = """
Read the summary and evaluate its fluency based on the given criteria. Assign a fluency score from 1 to 3.
"""


def get_geval_score(
    criteria: str, steps: str, document: str, summary: str, metric_name: str
):
    prompt = EVALUATION_PROMPT_TEMPLATE.format(
        criteria=criteria,
        steps=steps,
        metric_name=metric_name,
        document=document,
        summary=summary,
    )
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        max_tokens=5,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )
    return response.choices[0].message.content


evaluation_metrics = {
    "Relevance": (RELEVANCY_SCORE_CRITERIA, RELEVANCY_SCORE_STEPS),
    "Coherence": (COHERENCE_SCORE_CRITERIA, COHERENCE_SCORE_STEPS),
    "Consistency": (CONSISTENCY_SCORE_CRITERIA, CONSISTENCY_SCORE_STEPS),
    "Fluency": (FLUENCY_SCORE_CRITERIA, FLUENCY_SCORE_STEPS),
}

summaries = {"Summary 1": eval_summary_1, "Summary 2": eval_summary_2}

data = {"Evaluation Type": [], "Summary Type": [], "Score": []}

for eval_type, (criteria, steps) in evaluation_metrics.items():
    for summ_type, summary in summaries.items():
        data["Evaluation Type"].append(eval_type)
        data["Summary Type"].append(summ_type)
        result = get_geval_score(criteria, steps, excerpt, summary, eval_type)
        score_num = int(result.strip())
        data["Score"].append(score_num)

pivot_df = pd.DataFrame(data, index=None).pivot(
    index="Evaluation Type", columns="Summary Type", values="Score"
)
styled_pivot_df = pivot_df.style.apply(highlight_max, axis=1)
display(styled_pivot_df)